In [1]:
import warnings
warnings.filterwarnings('ignore')

# HTML Refresher
This part is based on chapter 11 of *Automate the Boring Stuff with Python* by Al Sweigart

HTML files are plain text files containing *tags*, which are words enclosed in angle brackets. Tags tell the browser how to format the web page. A starting tag and closing tag can enclose some text to form an element. The text (or inner HTML) is the content between the starting and closing tags.

There are many different tags in HTML. Some of these tags have extra properties in the form of attributes within the angle brackets. For example, the `<a>` tag encloses text that should be a link.

Some elements have an `id` attribute that is used to uniquely identify the element in the page. You will often instruct your programs to seek out an element by its id attribute, so figuring out an element’s id attribute using the browser’s developer tools is a common task in writing web scraping programs.

In [2]:
%%bash
cat << EOF > data/example.html
<!DOCTYPE html>
<html>
<head>
<title>Hello!</title>
</head>
<body>
<h1>Hello World!</h1>
You are extremely welcome!<br>
<br>
The <a href=\"https://github.com/datsoftlyngby/dat4sem2019spring-python-materials\">Lecture Notes</a>.<br>
<div>
<p>paragraph 1</p>
<p>and paragraph 2: <span id="span01">This is span 1</span id="span02"><span id="span03">Second span element</span>
<span class="red_border">Here is the third span</span>
</p>
</div>
</body>
</html>
EOF

In [3]:
!cat data/example.html

<!DOCTYPE html>
<html>
<head>
<title>Hello!</title>
</head>
<body>
<h1>Hello World!</h1>
You are extremely welcome!<br>
<br>
The <a href=\"https://github.com/datsoftlyngby/dat4sem2019spring-python-materials\">Lecture Notes</a>.<br>
<div>
<p>paragraph 1</p>
<p>and paragraph 2: <span id="span01">This is span 1</span id="span02"><span id="span03">Second span element</span>
<span class="red_border">Here is the third span</span>
</p>
</div>
</body>
</html>


# View a Page's HTML Sources

Here, I will only describe how to use Firefox' development features.

To view a page's sources right click on it and choose **View page source** which opens a new tab with the HTML sources.

<img src="images/view_source_small.png" width="500"> 

In Firefox, you can bring up the Web Developer Tools Inspector by pressing `CTRL-SHIFT-C` on Windows and Linux or by `CMD-OPTION-C` on OS X.

<img src="images/inspector_small.png" width="600"> 

# Parsing HTML with BeautifulSoup

BeautifulSoup is a module for parsing and extracting information from HTML sources. The module’s name is bs4. In case it is not already installed on your machine:
- install it with `pip install beautifulsoup4`. While beautifulsoup4 is the name used for installation, 
- to import BeautifulSoup you have to use `import bs4`.

According to its documentation (https://www.crummy.com/software/BeautifulSoup/) *"Beautiful Soup parses anything you give it, and does the tree traversal stuff for you. You can tell it "Find all the links", or "Find all the links of class externalLink", or "Find all the links whose urls match "foo.com", or "Find the table heading that's got bold text, then give me that text.""*

## Creating a BeautifulSoup Object from a Local HTML File

- The `bs4.BeautifulSoup()` function needs to be called with a string containing the HTML it will parse. 
- The `bs4.BeautifulSoup()` function returns is a `BeautifulSoup` object.

You can load a local HTML file and pass a file object to `bs4.BeautifulSoup()`.

In [4]:
import bs4

with open('./data/example.html') as f:
    example_html = f.read()
    
soup = bs4.BeautifulSoup(example_html)
print(type(soup))
print(soup.prettify())

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>
<html>
 <head>
  <title>
   Hello!
  </title>
 </head>
 <body>
  <h1>
   Hello World!
  </h1>
  You are extremely welcome!
  <br/>
  <br/>
  The
  <a href='\"https://github.com/datsoftlyngby/dat4sem2019spring-python-materials\"'>
   Lecture Notes
  </a>
  .
  <br/>
  <div>
   <p>
    paragraph 1
   </p>
   <p>
    and paragraph 2:
    <span id="span01">
     This is span 1
    </span>
    <span id="span03">
     Second span element
    </span>
    <span class="red_border">
     Here is the third span
    </span>
   </p>
  </div>
 </body>
</html>



## Creating a BeautifulSoup Object from a Remote HTML File



In [5]:
import bs4
import requests


r = requests.get('https://github.com/datsoftlyngby/dat4sem2020spring-python')
r.raise_for_status()
soup = bs4.BeautifulSoup(r.text, 'html.parser')

print(soup.prettify()[:1500])

<!DOCTYPE html>
<html data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="https://github.githubassets.com" rel="dns-prefetch"/>
  <link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
  <link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
  <link href="https://avatars.githubusercontent.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-b03f3373f0e7.css" integrity="sha512-sD8zc/Dn1lguLc4FdbeVhD2gfxFCTXGr/y+MpJG3oX1vkAyaFf/7BHPtvv1dAHA0KG4bDEW3Ex1EPrzGTMtN2Q==" media="all" rel="stylesheet">
   <link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-574f3bda934d.css" integrity="sha512-V0872pNNqPTaPQFzdwX9tHIzJvB4F1Foi6VYDciZAMo/qHgBC8PvN2d/

## Finding an Element with the `select()` Method

You can retrieve HTML elements from a `BeautifulSoup` object by calling the `select()` method and passing a string of a CSS selector for the element you are looking for. Selectors are like regular expressions: They specify a pattern to look for, in this case, in HTML pages instead of general text strings.

Common CSS selector patterns include:

  * `soup.select('div')` ... selects all elements named `<div>`
  * `soup.select('#lecturer')`  ... selects the element with an id attribute of author
  * `soup.select('.notice')` ... selects all elements that use a CSS class attribute named notice
  * `soup.select('div span')` ... selects all elements named `<span>` that are within an element named `<div>`
  * `soup.select('div > span')` ... selects all elements named `<span>` that are directly within an element named `<div>`, with no other element in between
  * `soup.select('input[name]')` ... selects all elements named `<input>` that have a name attribute with any value
  * `soup.select('input[type="button"]')` ... selects all elements named `<input>` that have an attribute named type with value button
  
See more in the documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors

In [6]:
import bs4


with open('./data/example.html') as f:
    example_html = f.read()

soup = bs4.BeautifulSoup(example_html, 'html.parser')

elems = soup.select('body')

#print(soup.prettify())
print('1: return type of select()',type(elems))
print('2: length of the returned list',len(elems))
print('3: type of elements in the list',type(elems[0]))
print('4: get text from the element',elems[0].getText()[:40])
print('5: string representation of an element: ',str(elems[0]))
print('6: the attributes of the element: ',elems[0].attrs)
elements = soup.select('div > p > span')
for element in elements:
    print(element.attrs)
print(soup.title.text)

1: return type of select() <class 'bs4.element.ResultSet'>
2: length of the returned list 1
3: type of elements in the list <class 'bs4.element.Tag'>
4: get text from the element 
Hello World!
You are extremely welcome!
5: string representation of an element:  <body>
<h1>Hello World!</h1>
You are extremely welcome!<br/>
<br/>
The <a href='\"https://github.com/datsoftlyngby/dat4sem2019spring-python-materials\"'>Lecture Notes</a>.<br/>
<div>
<p>paragraph 1</p>
<p>and paragraph 2: <span id="span01">This is span 1</span><span id="span03">Second span element</span>
<span class="red_border">Here is the third span</span>
</p>
</div>
</body>
6: the attributes of the element:  {}
{'id': 'span01'}
{'id': 'span03'}
{'class': ['red_border']}
Hello!


In [7]:
p_elems = soup.select('p')

for el in p_elems:
    print(str(el))
    print(el.getText())
    print('------------')

<p>paragraph 1</p>
paragraph 1
------------
<p>and paragraph 2: <span id="span01">This is span 1</span><span id="span03">Second span element</span>
<span class="red_border">Here is the third span</span>
</p>
and paragraph 2: This is span 1Second span element
Here is the third span

------------


## Getting Data from an Element’s Attributes

The `get()` method for Tag objects makes it simple to access attribute values from an element. The method is passed a string of an attribute name and returns that attribute’s value.

In [8]:
{'id': 'lecturer'}.get('id')

'lecturer'

In [9]:
import bs4

with open('./data/example.html') as f:
    example_html = f.read()
    
soup = bs4.BeautifulSoup(example_html, 'html.parser')
# soup.find_all?
span_elem = soup.select('span')[0]
print(str(span_elem))
print(span_elem.get('id'))
print(span_elem.get('some_nonexistent_addr') == None)
print(span_elem.attrs)

<span id="span01">This is span 1</span>
span01
True
{'id': 'span01'}


### What is the difference between the `select` and the `find`/`find_all` functions?

You are not the first ones wondering about this... See:
https://stackoverflow.com/questions/38028384/beautifulsoup-is-there-a-difference-between-find-and-select-python-3-x#38033910

# Example Scraping Events from a Page


Ususally, you will use web scraping to collect information, which you cannot gather otherwise. 
For example, let's imagine we want to do some statistics about:
- concerts in Copenhagen, 
- their start times and 
- their door prices.

Since we cannot find an API or any other open dataset, we decide to scrape the publicly available homepage www.kultunaut.dk, 

The website lists all possible events in Denmark. 
Concerts in Copenhagen are for example accessible here: 
- http://www.kultunaut.dk/perl/arrlist/type-nynaut/UK?showmap=&Area=Kbh.+og+Frederiksberg&periode=&Genre=Musik

**OBS** Many web pages are not built to support high traffic or they exlicitely discourage automatic access. Keep this in mind when writing your scraping tool.
- from time import sleep
- sleep(3) # sleep 3 seconds


Considering our example:
- we have to first figure out how many events there are at all. 
- We need this information, as events are given paginated, i.e., twenty events per page.
- The link given above only returns the link to the first page with the first twenty events. 
- Out of the total amount of events we can generate the URLs for the subsequent results.

In [13]:
kultunaut_url = 'https://www.kultunaut.dk/perl/arrlist/type-nynaut/UK?showmap=&Area=Kbh.+og+Frederiksberg&ArrStartdato=16%2F3+2021&ArrSlutdato=15%2F4+2021&Genre=Musik'
kultunaut_url2 = 'https://www.kultunaut.dk/perl/arrlist/type-nynaut/UK?showmap=&Area=Kbh.+og+Frederiksberg&ArrStartdato=16%2F3+2021&ArrSlutdato=15%2F4+2021&Genre=Musik&Startnr={}'
kultunaut_2022e = 'https://www.kultunaut.dk/perl/arrlist/type-nynaut/UK?showmap=&Area=Kbh.+og+Frederiksberg&nearmeradius=2000&ArrStartdato=15%2F3+2022&ArrSlutdato=15%2F4+2022&Genre=Musik'

In [14]:
r = requests.get(kultunaut_2022e)

r.raise_for_status()
soup = bs4.BeautifulSoup(r.text, 'html.parser')

res = soup.select('.result-count > strong:nth-child(2)')[0].text
no_of_events = res
print('no of events: {}'.format(no_of_events))


elems = soup.findAll('div',{'class':'arr-genre'})
print('Div with class arr-genre',elems[0])
print();print()
# annother notation
elems = soup.select('div[class=arr-genre] > h3 > strong')
print(elems)
#print(b_el)

no of events:  in Cph. and Frederiksberg  between 15 Mar and 15 Apr 2022
Div with class arr-genre <div class="arr-genre">
<span class="genre_cat notranslate">
          Music
          
        </span>
<h3><strong>Morgensang med Frederik Teglberg Damm</strong></h3>
</div>


[<strong>Morgensang med Frederik Teglberg Damm</strong>, <strong>Morgensang i Lundehus kirke</strong>, <strong>Sangarrangement</strong>, <strong>New Joy Gospel Choir</strong>, <strong>Alsang - fællessang i Kastelskirken</strong>, <strong>César Franck 200 år - samlede orgelværker I</strong>, <strong>Metroscope</strong>, <strong>The Magical Music of Harry Potter - live in concert</strong>, <strong>The Magical Music of Harry Potter - live in concert</strong>, <strong>Science &amp; Cocktails: David Wengrow</strong>, <strong>Uffe Lorenzen</strong>, <strong>Madrugada - european Tour 2022 - UDSOLGT</strong>]


In [ ]:
div.product:nth-child(1) > a:nth-child(1) > div:nth-child(2) > div:nth-child(1) > h3:nth-child(2) > strong:nth-child(1)

### Looking at the browser inspector pane:

<img src="images/inspect_element.png" width="500">

We can see that the desired element is hiding in a structure like: a b-tag inside a h3-tag inside a td-tag or:
- `('td h3 b')`

In [ ]:
# use select with css-selectors rather than find_all
import bs4
import requests
html = requests.get(kultunaut_url)
txt = html.text
soup = bs4.BeautifulSoup(txt, 'html.parser')
events = soup.select('div[class=arr-genre] > h3 > strong')
for e in events:
    print(e.getText())

In [ ]:
# Get all the links in a document
import requests
from bs4 import BeautifulSoup
gov = requests.get('https://analytics.usa.gov')
soup = BeautifulSoup(gov.text, 'lxml')
print(soup.prettify()[:100])
print('------------------------')
for link in soup.find_all('a'):
    if not link.get('href').startswith('https'):
        continue
    print(link.get('href'))

Now, we can scrape the events per page. Observe, that now, our `base_url` http://www.kultunaut.dk/perl/arrlist/type-nynaut/UK?Startnr={}&showmap=&Area=Kbh.%20og%20Frederiksberg&periode=&Genre=Musik& has a placeholder for the paginated results (`Startnr=`).

Consequently, we scrape each page separately, see the function on the next slide: `scrape_events_per_page`. From examining the page's source code, we know that events are all given as table entries with a corresponding header. We iterate over each of the table cells and extract the strings for dates and prices if they exist.

In [ ]:
from tqdm import tqdm

    
def scrape_events_per_page(url):
    """
    returns:
        A list of tuples of strings holding title, place, date, and price
        for concerts in Copenhagen scraped from Kulturnaut.dk
    """
    r = requests.get(url)
    r.raise_for_status()

    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    event_cells = soup.find_all('div', {'class' : 'arr-info'})
    #print('size',len(event_cells))
    print(event_cells[0])
    scraped_events_per_page = []
    
    for event_cell in event_cells:
        try:
            title = event_cell.select('h3 > strong')[0].text
            rest = event_cell.find('time').text.split(',')
            try:
                place = rest[1]
            except:
                place = ''
            try:
                date = rest[0]
            except:
                date = ''
        except Exception as e:
            print(e)
            continue
            
        scraped_events_per_page.append((title, place, date))
        
    return scraped_events_per_page


scraped_events = []
indexes = list(range(1, int(no_of_events), 12))
indexes[0] = 0


for idx in tqdm(indexes):
    scrape_url = kultunaut_url2.format(idx)
    #print(scrape_url)
    scraped_events += scrape_events_per_page(scrape_url)

In [ ]:
scraped_events[:20]

### How to Extract Dates and Prices from Strings.

Remember, the raw data, which we extracted from the web pages is all of type `str`. To do statistics about possible correlation of start times and entry fees, we need to convert the corresponding tuple fields into datetimes and integers respectively.


Since dates given on the web do not necessarily conform to standardized time formats, we can apply the `dateparser` (https://pypi.python.org/pypi/dateparser) module, which tries to parse arbitrary strings into datetimes.

You can read more about the module and its capabilities https://dateparser.readthedocs.io/en/latest/.

In [ ]:
%%bash
#pip install dateparser

In [ ]:
from tqdm import tqdm
import re
from dateparser import parse
from datetime import datetime


def get_dates_and_prices(scraped_events):
    """
    NO LONGER WORKS WELL WITH KULTUNAUT website after they changes layout and hid the prices behind a js function.
    Cleanup the data. Get price as integer and date as date.
    
    returns:
        A two-element tuple with a datetime representing the start 
        time of an event and an integer representing the price in Dkk.
    """

    price_regexp = r"(?P<price>\d+)" #initial ? is a lookbehind. r() r is for raw text, P<some pattern> is to give a pattern name to refer to. \d is numeric digit, + is for 1 or more.

    data_points = []
    three_at_night = datetime.now().replace(hour=3, minute=0, second=0, microsecond=0).time()
    for event_data in tqdm(scraped_events):
        title_str, place_str, date_str, price_str = event_data
        
        if 'Free admission' in price_str:
            price = 0
        else:
            m = re.search(price_regexp, price_str) # m is the Match object returned from re.search (might be None)
            try:
                price = int(m.group('price')) # if price can be converted to int then we do it else return 0.
            except:
                price = 0

        date_str = date_str.strip().strip('.')
        if '&' in date_str:
            date_str = date_str.split('&')[0]
        if '-' in date_str:
            date_str = date_str.split('-')[0]
        if '.' in date_str:
            date_str = date_str.replace('.', ':')
        
        date = parse(date_str)
        if date and date.time() > three_at_night:
            data_points.append((date, price))
            
    return data_points

def get_dates(scraped_events):
    """
    Cleanup the data. Get date as date.
    
    returns:
        A datetime representing the start 
        time of an event.
    """
    three_at_night = datetime.now().replace(hour=3, minute=0, second=0, microsecond=0).time()
    dates = []
    for event_data in tqdm(scraped_events):
        title_str, place_str, date_str = event_data
        
        date_str = date_str.strip().strip('.')
        if '&' in date_str:
            date_str = date_str.split('&')[0]
        if '-' in date_str:
            date_str = date_str.split('-')[0]
        if '.' in date_str:
            date_str = date_str.replace('.', ':')
        
        date = parse(date_str)
        if date and date.time() > three_at_night:
            dates.append(date)
    return dates

dates = get_dates(scraped_events)
print(dates)

In [ ]:
dates[10:20]

## Scraping Images from a Page

In the following code you will use Beautiful Soup to extract all links to images, which are in `img` tags on a web page.

In [ ]:
import bs4
import os
import sys
import requests
import shutil


def collect_img_links(url):
    """based on a url returns a list of image links contained in the requested page"""
    r = requests.get(url)
    r.raise_for_status()
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #print(soup.select('img'))
    return [img.get('src') for img in soup.select('img') 
            if img.get('src') and img.get('src').startswith('http')]


def download_imgs(links, out_folder="./data/test/"):
    """download all images from a list of image links. 
    Requires a folder named: test to be there"""
    img_no = 0
    for l in links:
        img_no += 1
        r = requests.get(l, stream=True)
        with open(out_folder+'img'+str(img_no)+'.jpg', 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)     
        
links = collect_img_links('https://www.google.dk/search?site=&tbm=isch&source=hp&biw=1163&bih=812&q=minions&oq=minions')
print(links)
download_imgs(links)

## 01 Exercise: Writing a Simple Web Crawler

1. Write a simple web scraper that can capture all images in a document (like: https://www.cphbusiness.dk/). 
2. Write to a .md file in a format so all images are shown when markdown is pasted and executed in a cell
3. Extend the web scraper to find all links (`<a>` elements) in the document.
4. If you have time, let the crawler find all links in the linked documents as well (so we get one more level of the hypertext graph). Use threads if helpfull.


In case a page returns a status code, which is not `200` we just disregard this page. See https://en.wikipedia.org/wiki/List_of_HTTP_status_codes for more detailes on the various HTTP status codes.


In [ ]:
def scrape_links(from_url, for_depth, all_links={}):
    # TODO: Implement code for websraper
    # return dict(key=url, value=list of outgoing urls)
    pass


start_url = 'https://www.version2.dk/artikel/google-deepmind-vi-oeger-sikkerheden-mod-misbrug-sundhedsdata-1074452'

link_dict = scrape_links(from_url=start_url, for_depth=2)

The web crawler that you wrote above is perhaps not the most performant. If you are interested in more web scraping and application of crawlers have a look at the `scrapy` module (https://scrapy.org)